In [97]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [98]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_from_hip.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [99]:
model = pin.buildModelFromUrdf(urdf_filename)
visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)
collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)
data = model.createData()

In [100]:
#list of joitns to lock
joints_to_lock=['right_foot_shin','right_shin_thigh','right_thigh_body','right_thigh_body_lateral']

IDs=[]

for joint_name in joints_to_lock:
    if model.existJointName(joint_name):
        IDs.append(model.getJointId(joint_name))
print(IDs)

initial_joint_config=np.array([0,0,0,0,0,0,0,0])

model_reduced, geom_models_reduced=pin.buildReducedModel(model,list_of_geom_models=[visual_model,collision_model],
    list_of_joints_to_lock=IDs,
    reference_configuration=initial_joint_config)
visual_model_reduced,collision_model_reduced=geom_models_reduced[0],geom_models_reduced[1]

[1, 2, 3, 4]


In [101]:
data_reduced=model_reduced.createData()
geom_data = pin.GeometryData(collision_model_reduced)

In [102]:
print('joints to lock (only ids):', IDs)
print('reduced model: dim=' + str(len(model_reduced.joints)))
print('-' * 30)

joints to lock (only ids): [1, 2, 3, 4]
reduced model: dim=5
------------------------------


In [103]:
visualizer = MeshcatVisualizer(model_reduced, collision_model_reduced, visual_model_reduced)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7018/static/


In [104]:
q_red=pin.neutral(model_reduced)
print(q_red)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX=20000
JOINT_ID = 4

oMdes = pin.SE3(np.eye(3), np.array([0.092, 0.01, 0.01])) #think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
# R_des = np.array(
#     [
#         [0.980062, 0.0029636, -0.198672],
#         [-0.0030271, 0.999995, -1.58699e-05],
#         [0.198671, 0.000616952, 0.980066],
#     ]
# )
# oMdes = pin.SE3(
#     R_des, np.array([0.147782, -3.96171e-06, 0.029025])
# )
i = 0

visualizer.display(q_red)

q=pin.neutral(model)


[0. 0. 0. 0.]


In [105]:
pin.forwardKinematics(model_reduced,data_reduced,q_red)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model_reduced.names, data_reduced.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

print(data_reduced.oMi[4])

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.076  0.000  0.290
body_left_thigh          :  0.076  0.000  0.290
left_thigh_shin          :  0.071  0.000  0.130
left_shin_foot           :  0.092 -0.000  0.000
  R =
    0.999995    0.0031853 -2.53654e-06
  -0.0031853     0.999995  4.03983e-09
 2.53654e-06  4.03983e-09            1
  p =       0.0925 -2.70751e-05  2.15606e-08



In [106]:
visualizer.viewer.jupyter_cell()

In [107]:
while True:
    pin.forwardKinematics(model_reduced, data_reduced, q_red)
    
    iMd = data_reduced.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model_reduced, data_reduced, q_red, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot_red = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q_red = pin.integrate(model_reduced, q_red, q_dot_red * DT)
    
    # #Collision check
    # pin.updateGeometryPlacements(
    #     model_reduced, data_reduced, collision_model_reduced, geom_data, q_red
    # )  # updates geom_data by reference

    # # an active pair is a pair of bodies in a joint considered for collision
    # collision_detected = pin.computeCollisions(
    #     model_reduced, data_reduced, collision_model_reduced, geom_data, q_red, True
    # )  # this returns data,geom_data. polymorphic function

    # if not collision_detected:
    #     for idx in range(model_reduced.nq):
    #         # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
    #         q_red[idx] = max(
    #             model_reduced.lowerPositionLimit[idx],
    #             min(q_red[idx], model_reduced.upperPositionLimit[idx]),
    #         )
    
    
    visualizer.display(q_red)
    print(norm_err)
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q_red.flatten().tolist())
print("\nfinal error: %s" % err.T)



0.014523706449214257
0.014313953864722949
0.014114695893035477
0.01392530324505971
0.013745208870885784
0.013573898739344064
0.01341090409997614
0.013255794999221313
0.01310817485511302
0.012967675923248878
0.012833955511561633
0.01270669282284439
0.012585586322429815
0.012470351544241785
0.012360719261967424
0.012256433963629229
0.012157252577653406
0.012062943406865466
0.01197328523391248
0.011888066567597139
0.011807085004668505
0.011730146685891923
0.011657065828824106
0.011587664322757773
0.011521771373855244
0.01145922319063555
0.011399862701777588
0.011343539299706641
0.011290108604683389
0.011239432245161995
0.011191377651044537
0.011145817857176167
0.011102631315011862
0.01106170171086428
0.011022917789533432
0.010986173182434514
0.010951366239589129
0.010918399865046243
0.010887181355455015
0.010857622241630407
0.010829638133042292
0.010803148565225847
0.010778076850158113
0.010754349929673626
0.01073189823201446
0.010710655531615588
0.010690558812229587
0.010671548133489299


In [108]:
pin.forwardKinematics(model_reduced,data_reduced,q_red)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model_reduced.names, data_reduced.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.076  0.000  0.290
body_left_thigh          :  0.076  0.000  0.290
left_thigh_shin          :  0.070  0.043  0.136
left_shin_foot           :  0.092  0.010  0.010


In [109]:
result_deg=[np.rad2deg(i) for i in q_red]
print(result_deg)

[0.029030379947827843, -15.515134737142471, 30.122951200526515, 14.607830388692136]
